Gemini API quickstart

In [3]:
pip install -q -U google-genai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.30.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.5 which is incompatible.
streamlit 1.30.0 requires rich<14,>=10.14.0, but you have rich 14.0.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from google import genai

GEMINI_API_KEY = """

client = genai.Client(api_key=GEMINI_API_KEY)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Explain how AI works in a few words",
)

print(response.text)

AI learns patterns from data to make decisions or predictions.


In [ ]:
from google import genai

GEMINI_API_KEY = ""

client = genai.Client(api_key=GEMINI_API_KEY)

response = client.models.generate_content(
    model="gemini-2.5-flash", contents="Explain how AI works in a few words"
)
print(response.text)

AI learns patterns from data to make predictions or decisions.


In [7]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GEMINI_API_KEY)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Explain how AI works in a few words",
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0) # Disables thinking
    ),
)
print(response.text)

AI works by finding patterns in data.


In [ ]:
from google import genai

GEMINI_API_KEY = ""

client = genai.Client(api_key=GEMINI_API_KEY)
prompt = "Explain the concept of Occam's Razor and provide a simple, everyday example."
response = client.models.generate_content(
    model="gemini-2.5-pro",
    contents=prompt
)

print(response.text)

Of course! Here is an explanation of Occam's Razor with a simple, everyday example.

### What is Occam's Razor?

Occam's Razor is a problem-solving principle or a "rule of thumb" that can be summed up as:

**"Among competing hypotheses, the one with the fewest assumptions should be selected."**

In simpler terms, it means that **the simplest explanation is usually the right one.**

It's important to understand what "simplest" means in this context. It doesn't mean the easiest explanation to understand. It means the explanation that requires the fewest new or unproven ideas to make it work. Each assumption you have to make is another point where the explanation could be wrong.

The principle is named after William of Ockham, a 14th-century philosopher and theologian, and it serves as a powerful guide in science, philosophy, and everyday reasoning.

**Key Points to Remember:**
*   It's not an unbreakable law; it's a guideline.
*   It doesn't say that the more complex theory is *definitel

In [1]:
import langchain

In [ ]:
pip install sentence-transformers pymilvus langchain

In [8]:
from sentence_transformers import SentenceTransformer

# Load pre-trained embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Sample Walmart content (e.g., product or policy document)
docs = [
    "Return policy: Electronics must be returned within 45 days.",
    "Product: ₹999 kids backpack with cartoon print and zip pockets.",
    "Order placed after 8 PM will ship next day from Pune warehouse."
]

# Generate embeddings
embeddings = model.encode(docs)

In [ ]:
cluster_endpoint = ".zilliz.com"
api_key = " "

In [9]:
from pymilvus import connections

connections.connect(
    uri=cluster_endpoint,
    token=api_key,
    secure=True
)

print("✅ Connected to Milvus Cloud!")

✅ Connected to Milvus Cloud!


In [12]:
docs = [
    "Walmart allows electronics returns within 30 days with a receipt.",
    "Suppliers must complete onboarding forms before shipment approval.",
    "Walmart Plus members get free shipping on all orders.",
    "Returns on perishable items are not accepted beyond 7 days.",
    "Employees can access HR policies via the internal portal."
]

In [13]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(docs)

In [14]:
from pymilvus import connections

connections.connect(
    uri=cluster_endpoint,
    token=api_key,
    secure=True
)
print("✅ Connected to Milvus Cloud!")

✅ Connected to Milvus Cloud!


In [15]:
from pymilvus import Collection, FieldSchema, CollectionSchema, DataType

fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="content", dtype=DataType.VARCHAR, max_length=512),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384),
]
schema = CollectionSchema(fields)
collection = Collection("walmart_docs", schema)

In [16]:
insert_data = [
    [doc for doc in docs],
    [vec.tolist() for vec in embeddings]
]
collection.insert(insert_data)

(insert count: 5, delete count: 0, upsert count: 0, timestamp: 458909241887424515, success count: 5, err count: 0, cost: 2)

In [17]:
collection.create_index(
    field_name="embedding",
    index_params={"index_type": "AUTOINDEX", "metric_type": "COSINE"}
)
collection.load()

In [18]:
query = "What is the return policy for electronics?"
query_embedding = model.encode([query]).tolist()

results = collection.search(
    data=query_embedding,
    anns_field="embedding",
    param={"metric_type": "COSINE", "params": {"nprobe": 10}},
    limit=1,
    output_fields=["content"]
)

for hit in results[0]:
    print("💡 Top Match:", hit.entity.get("content"), "→ Score:", hit.distance)

💡 Top Match: Return policy: Electronics must be returned within 45 days. → Score: 0.7825798988342285
